In [1]:
import csv
import re

In [2]:
with open('KEGG_BRITE_br/br08002.keg','rb') as f:
    reader = csv.reader(f,delimiter = '\t')
    for row in reader:
        if row == ['A<b>Antibiotics</b>']:
            print row

essentially...want to search for a given compound, find its group. Then can plot by group either as the most specific group or all the way up the hierarchy to the broadest group

In [3]:
import cPickle as cpk

In [4]:
WorkingFile = cpk.load(open('exportCOnumbers.csv.pickle','r'))

In [5]:
WorkingFile['C14666']

{'Modules': [],
 'Name': 'C14666',
 'Reaction': {},
 'Related CO': [],
 'Related KO': []}

In [40]:
import pandas as pd
import numpy as np

In [7]:
briteFile=pd.read_table('KEGG_BRITE_br/br08002.keg',sep = '\t',skiprows=7)

In [8]:
briteFile.head(10)

,!
0,A<b>FA Fatty acyls</b>
1,B FA01 Fatty Acids and Conjugates
2,C FA0101 Straight chain fatty acids
3,D C00058 Formic acid
4,D C00033 Acetic acid
5,D C00163 Propanoic acid
6,D C00246 Butanoic acid
7,D C00803 Pentanoic acid
8,D C01585 Hexanoic acid
9,D C17714 Heptanoic acid


In [9]:
briteFile.shape

(2803, 1)

In [10]:
briteFile.loc[8,:]

!    D      C01585  Hexanoic acid
Name: 8, dtype: object

In [11]:
m = briteFile.loc[8,:]
m

!    D      C01585  Hexanoic acid
Name: 8, dtype: object

In [12]:
type(m)

pandas.core.series.Series

In [13]:
briteRE = re.compile(r'(^D)\s+(C\d{5})\s*(.*)$') #three groups: D, C#, text with name

In [14]:
m.index

Index([u'!'], dtype='object')

In [15]:
m['!']

'D      C01585  Hexanoic acid'

In [16]:
m.describe()

count                                1
unique                               1
top       D      C01585  Hexanoic acid
freq                                 1
Name: 8, dtype: object

In [17]:
m.item()

'D      C01585  Hexanoic acid'

In [18]:
m.ravel()

array(['D      C01585  Hexanoic acid'], dtype=object)

In [19]:
m.unstack

<bound method Series.unstack of !    D      C01585  Hexanoic acid
Name: 8, dtype: object>

In [20]:
r = m.unstack

In [21]:
r

<bound method Series.unstack of !    D      C01585  Hexanoic acid
Name: 8, dtype: object>

In [22]:
m = briteFile[briteFile['!'].str.contains("C00163")]
m

,!
5,D C00163 Propanoic acid


so, now can I search back up the rows to find the C, B, and A compound information?

In [23]:
type(m)

pandas.core.frame.DataFrame

In [24]:
m.index

Int64Index([5], dtype='int64')

In [25]:
df = briteFile.copy(deep = True)

In [26]:
r = m.loc

In [27]:
r

In [32]:
briteFile = 'KEGG_BRITE_br/br08002.keg'
#britekoRE = re.compile(r'^[A-Z]\s+(K\d{5})\s*(.*)$') #example from kegg.py

readDlevel = re.compile(r'(^D)\s+(C\d{5})\s*(.*)$') #three groups: D, C#, text with name


In [33]:
my_dict = {}
#counter = 1
with open(briteFile) as f:
    for line in f:
        #print line
        m = briteRE.search(line)
        if m:
            cNumber = m.group(2)
            level = m.group(1)
            cName=m.group(3)
            small = {}
            small['level'] = level
            small['cName'] = cName
            my_dict[cNumber]=small


In [34]:
my_dict['C14662']

{'cName': '(E)-4-Nitrostilbene', 'level': 'D'}

In [120]:
briteFile = 'KEGG_BRITE_br/br08002.keg'
# textA = re.compile(r'(^A)\s*(.*)$')
textA = re.compile(r'(^A<b>)(.+)(</b>)\s*(.*)$')
textB = re.compile(r'(^B)\s*(.*)$')
textC = re.compile(r'(^C)\s*(.*)$')
textD = re.compile(r'(^D)\s+(C\d{5})\s*(.*)$') #three groups: D, C#, text with name
text_cNumber = re.compile(r'(\d*)+(C\d{5})\s*(.*)$')

In [121]:
forBrite.head(5)

,cNumber,A,B,C,D,wholeThing
8,NaN,FA Fatty acyls,NaN,NaN,NaN,A<b>FA Fatty acyls</b>\n
9,NaN,FA Fatty acyls,FA01 Fatty Acids and Conjugates,NaN,NaN,B FA01 Fatty Acids and Conjugates\n
10,NaN,FA Fatty acyls,FA01 Fatty Acids and Conjugates,FA0101 Straight chain fatty acids,NaN,C FA0101 Straight chain fatty acids\n
11,C00058,FA Fatty acyls,FA01 Fatty Acids and Conjugates,FA0101 Straight chain fatty acids,C00058,D C00058 Formic acid\n
12,C00033,FA Fatty acyls,FA01 Fatty Acids and Conjugates,FA0101 Straight chain fatty acids,C00033,D C00033 Acetic acid\n


logic not quite right below...but getting close. Stopping for night.

In [122]:
from IPython.core.debugger import Tracer
forBrite = pd.DataFrame(columns = ['cNumber','A','B','C','D','wholeThing'])

#this relies on the fact that the rows are in order: A, with B subheadings, then C subheadings, finally D subheadings
setA = []
idxA = []

setB = []
setC = []
setD = []

with open(briteFile) as f:
    for idx,line in enumerate(f):
        mA = textA.search(line) 
        mB = textB.search(line) 
        mC = textC.search(line) 
        mD = textD.search(line) 
        if mA:
            setA = mA.group(2)
            #house cleaning (probably c)
            idxA = idx
            forBrite.loc[idx,'A'] = setA
            forBrite.loc[idx,'wholeThing'] = line #using this as a double check for now
        elif mB:
            setB = mB.group(2)
            forBrite.loc[idx,'A'] = setA
            forBrite.loc[idx,'B'] = setB
            forBrite.loc[idx,'wholeThing'] = line
        elif mC:
            setC = mC.group(2)
            forBrite.loc[idx,'A'] = setA
            forBrite.loc[idx,'B'] = setB
            forBrite.loc[idx,'wholeThing'] = line
            
            #Look for a cNumber               
            test = text_cNumber.search(line)
            if test:
                #put the cNumber into the right place
                forBrite.loc[idx,'cNumber'] = test.group(2)
            else: 
                forBrite.loc[idx,'C'] = setC
            
        elif mD:
            forBrite.loc[idx,'A'] = setA
            forBrite.loc[idx,'B'] = setB
            forBrite.loc[idx,'C'] = setC
            forBrite.loc[idx,'wholeThing'] = line
            
            #look for the cNumber               
            test = text_cNumber.search(line)
            if test:
                #put the cNumber into the right place
                forBrite.loc[idx,'cNumber'] = test.group(2)
            else: 
                #not sure this will ever be anything, but put here for now
                forBrite.loc[idx,'D'] = mD.group(2)


In [123]:
forBrite

,cNumber,A,B,C,D,wholeThing
8,NaN,FA Fatty acyls,NaN,NaN,NaN,A<b>FA Fatty acyls</b>\n
9,NaN,FA Fatty acyls,FA01 Fatty Acids and Conjugates,NaN,NaN,B FA01 Fatty Acids and Conjugates\n
10,NaN,FA Fatty acyls,FA01 Fatty Acids and Conjugates,FA0101 Straight chain fatty acids,NaN,C FA0101 Straight chain fatty acids\n
11,C00058,FA Fatty acyls,FA01 Fatty Acids and Conjugates,FA0101 Straight chain fatty acids,NaN,D C00058 Formic acid\n
12,C00033,FA Fatty acyls,FA01 Fatty Acids and Conjugates,FA0101 Straight chain fatty acids,NaN,D C00033 Acetic acid\n
13,C00163,FA Fatty acyls,FA01 Fatty Acids and Conjugates,FA0101 Straight chain fatty acids,NaN,D C00163 Propanoic acid\n
14,C00246,FA Fatty acyls,FA01 Fatty Acids and Conjugates,FA0101 Straight chain fatty acids,NaN,D C00246 Butanoic acid\n
15,C00803,FA Fatty acyls,FA01 Fatty Acids and Conjugates,FA0101 Straight chain fatty acids,NaN,D C00803 Pentanoic acid\n
16,C01585,FA Fatty acyls,FA01 Fatty Acids and Conjugates,FA0101 Straight chain fatty acids,NaN,D C01585 Hexanoic acid\n
17,C17714,FA Fatty acyls,FA01 Fatty Acids and Conjugates,FA0101 Straight chain fatty acids,NaN,D C17714 Heptanoic acid\n
